# Statistic for AI and Data Science : Coursework 2
### Part 1: Data Preparation
### Part 2: Exploratory Analysis
### Part 3: Regression Modelling

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
%matplotlib inline

ModuleNotFoundError: No module named 'sklearn'

**Values of Categorical Variables** In the original data, the values of the categorical variables are represented as integers, with their meanings given in a data dictionary. In our dataset, these 'numeric codes' have been replaced with suitable names.

| Variable      |      Values            |
|:--------------|:-----------------------|
|District       | Each district has a unique number  |
|Toll           | Toll, Free                |
|Maintainer     | State, County, Town or City, Agency, Private, Railroad, Toll Authority, Military, Unknown |
|Urban          | Urban, Rural |
|Status         | Interstate, Arterial, Minor, Local |
|Historic       | Register, Possible, Unknown, Not historic |
|Service_under  | Other, Highway, Railroad, Pedestrian, Interchange, Building |
|Material       | Other, Concrete, Steel, Timber, Masonry |
|Design         | Other, Slab, Beam, Frame, Truss, Arch, Suspension, Movable, Tunnel, Culvert, Mixed |
|Scour_rating   | Unknown, Critical, Unstable, Stable, Protected, Dry, No waterway |
|Deck_rating    | *Rating*: NA, Excellent, Very Good, Good, Satisfactory, Fair, Poor, Serious, Critical, Failing, Failed |
|Superstr_rating| *Rating* |
|Substr_rating  | *Rating* |

### Part 1: Data Preparation

This report looks at some data from the US National Bridge Inspection maintained by the Federal Highways Agency (FHWA), part of the US Department of Transportation. The data comes from the [National Bridge Inspection](https://www.fhwa.dot.gov/bridge/nbi/ascii.cfm) section of the FHWA's web site. However, it has been greatly simplified.

**The aim of the Bridge Inspection programme is to check on the state of bridges so that necessary repairs can be carried out. If this is not done, a bridge can fail. The dataset has information about the bridges and the conditions

* This FHWA's database will cover bridges in the state of Texas. 
* As well as bridges, the FHWA's database covers tunnels.


#### The Below table provides all the initial data being used in order to create a more cleaner dataset to work with and interpret.

##### Removing **Outliers** and simplifying categories.

- Create and return an age variable from year.
- Remove historic bridges.
- Merge and remove small categories.
- Create current condition column from deck_rating, superstr_rating and substr_rating.

In [ ]:
rating_type = pd.CategoricalDtype(
    categories=['Failed', 'Failing', 'Critical', 'Serious', 'Poor', 'Fair', 
                'Satisfactory', 'Good', 'Very Good', 'Excellent', 'NA'], 
    ordered=True)

scour_type = pd.CategoricalDtype(
    categories=['Unknown', 'Critical','Unstable', 'Stable', 'Protected', 'Dry', 'No waterway'], 
    ordered=True)

types_dict = { 'Structure_id': str, 'District':'category', 'Toll':'category', 
              'Maintainer':'category', 'Urban':'category', 'Status':'category', 
              'Historic':'category', 'Service_under':'category', 'Material':'category', 
              'Design':'category', 
              'Deck_rating':rating_type, 'Superstr_rating':rating_type, 'Substr_rating':rating_type, 
              'Scour_rating':scour_type}

bridges = pd.read_csv('tx19_bridges_sample.csv', dtype = types_dict, index_col = 'Structure_id')
bridges  

In [ ]:
bridges["Historic"].value_counts()

The table below like the above table also now includes the **Age** Variable. This will allow for easy of understanding and clarity

In [ ]:
bridges = bridges.assign(Age = 2021 - bridges.Year)
bridges

The below table has a few amendments. The outliers have been removed which are the very old bridges i.e. the Historic ones. This will help to keep a cleaner more usable dataset. This has been taken from the variable variable **Historic** and taking out any data that is registered historic and 50 years old and over.

The reason for this is due to the fact that a bridge that are over 50 years old can be considered historic.
The [Taxas Historical Commission](https://www.thc.texas.gov/preserve/buildings-and-property) further elorates about the 50-year rule and how and why a particular texas bridge may be considered historic. 

There are many other bridges that are much older than 50 years old. However, they are very much still in use hence why bridges with an average daily use of 10 or less has also been removed from this set of data. Some very old bridges may have also gone through a refurbishment.

In [ ]:

bridges = bridges.drop(index = bridges.loc[(bridges.Age >= 50) & (bridges.Historic == "Register")].index)
bridges = bridges.drop(index = bridges.loc[(bridges.AverageDaily <= 10)].index)
bridges

In [ ]:
bridges["Historic"].value_counts()

For both bridge Design and Material, the Design data has been merged to create a tidier more easily usuable data. Frame, Movable and Suspension bridges will now be part of **Other**. While Masonry and Timber will also be part Other for Materail.
Design.

It can happen that some categorical features have quite a few values with a small number of occurrence. This can often be a problem during while attempting to carry out data analysis.

In [ ]:
def Design_ (row):
    if (row.Design ==  "Frame") : return "Other"
    if (row.Design ==  "Movable") : return "Other"
    if (row.Design ==  "Suspension") : return "Other"
    return row.Design

bridges.Design = bridges.apply(Design_, axis=1)
bridges

In [ ]:
bridges["Design"].value_counts()

In [ ]:
def Material_ (row):
    if (row.Material ==  "Masonry") : return "Other"
    if (row.Material ==  "Timber") : return "Other"
    return row.Material

bridges.Material = bridges.apply(Material_, axis=1)
bridges

In [ ]:
bridges["Material"].value_counts()

Another amendment that has been made to the data is to change the Deck, Superstructure and Substructure rating into numeric(integer) score rating starting from 0 - 9 with 0 as Failed and 9 as Excellent. Once the three scores/ratings have been created, a new varible called current condition will be made that will be made up of all the ratings totalled together.

This has allowed the three categorical variables to be made into one continuous variable.

In [ ]:
ratings = { "Failed": 0, "Failing": 1, "Critical": 2, "Serious": 3, "Poor": 4, "Fair": 5, "Satisfactory": 6, "Good" : 7, "Very Good": 8, "Excellent": 9
}

bridges = bridges.replace(ratings)

bridges["Current_Condition"] = bridges.Deck_rating + bridges.Superstr_rating + bridges.Substr_rating
bridges
                   

## Part 2: Exploratory Analysis

Relationship between predictor variables and target variables
- relationship between continuous variables
- relationship between continuous variables and categorical variables
- relationship between categorical variables
- relationship between predictor variables and target variable

Below is a range of data that has been pulled from the original bridges dataset. This group of data will be used to assess the relationship between a number of predictor variables being Age, Average daily use (vehicles), number of trucks that use the bridge daily, bridge material and bridge design. This will be assessed against the current condition.

Please pay particular attention to the Ratings and the current condition table this provides a clear idea of the condition of each bridge.

| Rating      |      Current Condition   |
|:--------------|:-----------------------|
|0     | Failed  |
|1 - 3     | Failing|
|4 - 6     | Critical |
|7 - 9   | Serious |
|10 - 12        |Poor|
|13 - 15       | Fair|
|16 - 18  | Satisfactory|
|19 - 21       |Good|
|22 - 24        | Very Good|
|25 - 27        | Excellent|

In [ ]:
bridges_five_pred = bridges.loc[:,["Age", "AverageDaily", "Trucks_percent", "Material", "Design", "Current_Condition"]]
bridges_five_pred

##### Relationship between continuous variables

Below displays the relationship between continuous variables


The histogram below ignores categorical variables and only includes continous variables i.e. data with numerical values which can be an infinite set of possible values.

The below **Age** histogram appears positively skewed and it appears that their are less older bridges than there are newer ones. The data spread is from about 2 years to 121 years and the greatest peak is around about 15 to 18 years. This shows that most bridges were built around this time.

The **AverageDaily** also appears positively skewed with the most common average daily being around about the 50 mark. This data ranges from about 11 to 543000. This wide range shows that there are bridges that can be very busy while others have few visits. 

It appear that the **Trucks_Percent** is very few and most bridges get 0 trucks that cross daily. 

Lastly we have the **Current_Condition** which is somewhat symmetrical with a mean of 20 a median of 21 and a mode of also 21. This shows that the the majority of texas bridge are in good condition.

In [ ]:
hist_bridges = bridges_five_pred.hist(color=["blue"], bins=50, figsize=(20,10))

Below is a table to show how each of the continous variables correlate. From a first glance it is clear there are no clear strong correlations between the variables. However, age and current condition seem to have a moderately negative correlation meaning there is a clear relationship between the two variables in question, but there's also a lot of randomness affecting one or both variables, or perhaps other variables affect the two variables in question, so the direct relationship isn't strong, but it certainly exists.



In [ ]:
bridges_five_pred.corr()

This two tables below shows some useful information such as the mean, median, min, max and mode. The average age of any Texas bridge is 39 years old while the most common age is 15 years this is mainly due to the fact that there are some very old bridges that are up to 121 years old. You may also notice an average daily use of 11841 while the max is 543000 which may have an impact on the overall mean especilly as the min is 11.

Another interesting find is the median, which is the middle (50%) point. It is quite useful to compare the median and the mean as this helps to identify any anomolies within the data. With Age this is due to having very old bridges in the dataset, AverageDaily has a bridge with 543000 crossings daily, Truck percent and current condition have not showed any massively clear anomolies as of yet.

In [ ]:
bridges_five_pred.describe()

In [ ]:
bridges_five_pred.mode()

The below heatmap would only include continous variables this is to show the correlation between each continuous variable. 

An absolute value of 1 indicates a perfect linear relationship which would be coloured in blue. A correlation close to 0 indicates non linear relationship between the variables which is coloured in white. If both variables tend to increase or decrease together, the coefficient is positive, and the line that represents the correlation slopes upward. When there appears to be a negative correlation the point would be coloured in red. 

In [ ]:
fig,ax = plt.subplots(1,1, figsize=(15,12))
sns.heatmap(bridges_five_pred.corr(), vmin=-1, vmax=1, cmap=sns.diverging_palette(20, 220, as_cmap=True), 
            annot=True, ax=ax, annot_kws={"size": 15})
_y = plt.yticks(rotation=0, fontsize=15)
_x = plt.xticks(rotation=45, fontsize=15)

The below scatter matrix is again showing the relation between the continous variables and was noticed previously no clear relation can be found as of yet between these continuous variables.

In [ ]:
_a = pd.plotting.scatter_matrix(bridges_five_pred, figsize=(16,16))

The first hexbin shows a clear dark point around about the 15 years mark with a current condition rating around about 22. This rating is pretty similar from bridges ages 70 up to the age of 2.

When looking at the Average daily and trucks percent. It is obvious that most vehicle travel over bridges that are in good condition.

In [ ]:
bridges_five_pred.loc[bridges_five_pred.Current_Condition < 30].plot(kind='hexbin',  x='Age', y='Current_Condition',  gridsize=20, sharex=False)
bridges_five_pred.loc[bridges_five_pred.Current_Condition < 30].plot(kind='hexbin',  x='AverageDaily', y='Current_Condition',  gridsize=20, sharex=False)
bridges_five_pred.loc[bridges_five_pred.Current_Condition < 30].plot(kind='hexbin',  x='Trucks_percent', y='Current_Condition',  gridsize=20, sharex=False)


##### Relationship between continuous variables and categorical variables

The below tables are known as pivot tables. This has been done to show the relationship between categorical and continuous variables.
The first table shows the relationship between Material and how it compares to the continuous variables.

One thing that was immediately noticable is the fact that there are not that many more average daily Concrete bridge crossings when compared to Steel bridge crossings. Although concrete bridge crossings are slightly more the current condition rating is more or less the same. With trust percent is quite obvious that trucks seem to travel more via a concrete bridge when compared to a Steel bridge.

In [ ]:
table1 = pd.pivot_table(bridges_five_pred, index=['Material'])
table1

The table2 shows that most crossings are made on Beam bridges but trucks tend to travel via Slab bridges. Notwithstanding, that the highest current condition rating comes from Arch bridges.

Another interesting factor to take note of is that Slab bridges are the oldest bridges with an average age of 57 years old.

In [ ]:
table2 = pd.pivot_table(bridges_five_pred, index=['Design'])
table2

The below bar charts are a visual representation of how the bridge material and bridge design compare of current condition. Beam is clearly the most common bridge design, while concrete is the most common bridge material. 

In [ ]:
fig, (a1, a2) = plt.subplots(2, 1, figsize = (8, 10))

bridges_five_pred.pivot_table(values='Current_Condition', index='Design', aggfunc='count').plot(
    ax=a1, kind='bar', log=True, legend=True)
bridges_five_pred.pivot_table(values='Current_Condition', index='Material', aggfunc='count').plot(
    ax=a2, kind='bar', log=True, legend=True)



You can see the distinctive connection between Design and the current condition ratings. It is not surprising but exciting to see the correlation between the two variables through the data collected.

In the data, you might notice the most common rating amongst all bridge design fall between a rating of around 20 to 22 with the exception of Arch where quite a few of their bridge designs fall in a rating of 24.

Arch has managed to maintain pretty good ratings with nothing less than 15 and ratings as high as 24 making it a good safe and well maintained design type. This can also be said about all bridge designs as all fall in the Satisfactory to Excellent section of the current condition ratings.

In [ ]:
pd.options.display.max_columns = None

curr_con_and_des = pd.crosstab(bridges_five_pred.Current_Condition, [bridges_five_pred.Design], normalize='all')
curr_con_and_des_tot = pd.crosstab(bridges_five_pred.Current_Condition, [bridges_five_pred.Design], normalize='all', margins=True)
des_and_curr_con = pd.crosstab(bridges_five_pred.Design, [bridges_five_pred.Current_Condition], normalize='all')

des_given_curr_con = pd.crosstab(bridges_five_pred.Current_Condition, [bridges_five_pred.Design], normalize='index')
curr_con_given_des = pd.crosstab(bridges_five_pred.Design, [bridges_five_pred.Current_Condition], normalize='index')

curr_con_and_des_tot.round(4) * 100
des_and_curr_con.round(4) * 100
des_given_curr_con.round(4) * 100
curr_con_given_des.round(4) * 100


It appears that Concrete has an higher most common current condition rating in comparison to Steel and Other. This could be due to a number of factors. Concrete could just be a better material for bridges, Steel may rust and deteriate with time or the concrete bridges may have been built more recently.

In [ ]:
pd.options.display.max_columns = None

curr_con_and_mat = pd.crosstab(bridges_five_pred.Current_Condition, [bridges_five_pred.Material], normalize='all')
curr_con_and_mat_tot = pd.crosstab(bridges_five_pred.Current_Condition, [bridges_five_pred.Material], normalize='all', margins=True)
mat_and_curr_con = pd.crosstab(bridges_five_pred.Material, [bridges_five_pred.Current_Condition], normalize='all')

mat_given_curr_con = pd.crosstab(bridges_five_pred.Current_Condition, [bridges_five_pred.Material], normalize='index')
curr_con_given_mat = pd.crosstab(bridges_five_pred.Material, [bridges_five_pred.Current_Condition], normalize='index')

curr_con_and_mat_tot.round(4) * 100
mat_and_curr_con.round(4) * 100
mat_given_curr_con.round(4) * 100
curr_con_given_mat.round(4) * 100

The below boxplot provides a graphical summary of the distribution of some samples. The boxplot shows the shape, central tendency, and variability of the data.

From observing the box plot it can be found that there are quite a few visible outliers. What this means is that there are data values that are far away from other data values which can also greatly affect the results of any analysis.

The visible outliers are due to the very low rated/failed bridges or even some very high rated bridges also which are in excellent condition

In [ ]:
fig, (a1, a2) = plt.subplots(2,1, figsize=(15,14))

bridges_five_pred.boxplot(column='Current_Condition', by='Material', ax=a1)
bridges_five_pred.boxplot(column='Current_Condition', by='Design', ax=a2)


# Make the plots a bit clearer
fig.suptitle('')
[a.set_title('') for a in [a1, a2]]
a1.set_ylabel('Current Condition')
a2.set_ylabel('Current Condition')


##### Relationship between categorical variables

The below table compares two categorical variables being material and design. It appears that Beam is the most common bridge design as it holds the highest percentage for all three bridge materials. Truss appears to be the least used design type followed by Arch.

It can also be observed that Slab bridges are only made from concretes and truss bridges are only made from Steel.


In [ ]:
des_and_mat = pd.crosstab(bridges_five_pred.Design, [bridges_five_pred.Material], normalize='all')
des_and_mat_tot = pd.crosstab(bridges_five_pred.Design, [bridges_five_pred.Material], normalize='all', margins=True)
mat_and_des = pd.crosstab(bridges_five_pred.Material, [bridges_five_pred.Design], normalize='all')

mat_given_des = pd.crosstab(bridges_five_pred.Design, [bridges_five_pred.Material], normalize='index')
des_given_mat = pd.crosstab(bridges_five_pred.Material, [bridges_five_pred.Design], normalize='index')

des_and_mat_tot.round(4) * 100
mat_and_des.round(4) * 100
mat_given_des.round(4) * 100
des_given_mat.round(4) * 100

The below Stacked bar charts is a great way to compare absolute values between bars.

The stacked chart has been configured to 100%; this enables a clear comparison of the relative values for each bar.

When looking at the probability of Material given Design, you can see below, concrete being the blue bar makes up the majority of all bridge material followed by steel.

The chart this also confirms the earlier statement about Slab bridges being made of Concrete and Truss bridges being made of Steel.

When observing the chart for probability of Design given Material Beam hold an overall majority as being the most common bridge design.


In [ ]:
fig,(a1, a2) = plt.subplots(2,1,figsize=(15,20), sharey=False, sharex=False)
fig.subplots_adjust(hspace=0.35)

mat_given_des.plot(kind='bar', subplots=False, ax=a1, rot=0, stacked=True)
a1.set_title('Probability of Material, given Design', fontsize=14)

des_given_mat.plot(kind='bar', subplots=False, ax=a2, rot=0, stacked = True)
a2.set_title('Probability of Design, given Material', fontsize=14)

For this final example that compares categorical variables, the below crosstab has been passed to a seaborn heatmap in order to visually summarize the data.

With the use of a heatmap, we can easily interpret the data. Fortunately, seaborn can take the output from the crosstab and visualize it:


In [ ]:
mat_given_des1 = pd.crosstab(bridges_five_pred.Material, [bridges_five_pred.Design], normalize='index')

fig,ax = plt.subplots(1,1, figsize=(12,10))
sns.heatmap(mat_given_des.round(4)*100, cmap=sns.light_palette('grey'), linewidths = 2,
            annot=True, ax=ax, annot_kws={"size": 13}, fmt='g')
ax.set_title('Superstructure Rating given Deck Rating', fontsize=14)

## Part 3: Regression Modelling

- You should record the R2 (coefficient of determination) and comment on the value.
- You should show and comment on the distribution of residuals (errors).
- You should use the regression coefficients to compare the influence of the different predictors.
- You should draw final conclusions at the end of this part of the analysis on the answers to the questions asked by the Texas Department of Transportation. You should include brief suggestions for further analysis

#### Below a linear regression has been solved. 

To minimise errors the NaN values have been dropped.

In [ ]:
bridges_five_pred1 = bridges_five_pred.dropna()
print(bridges_five_pred1)

Both Material and Design have been made into dummie variables to which will be represented as either 0/1. This will also allow these variables to be used to carry out a regression analysis. The first column will also be dropped in order to avoid correlation or collinearity

In [ ]:
Mat_D = pd.get_dummies(bridges_five_pred1.Material, drop_first=True)
Mat_D

In [ ]:
Des_D = pd.get_dummies(bridges_five_pred1.Design, drop_first=True)
Des_D

#### Below is a record of the R2 (coefficient of determination) and the comment attached

Current condition, Average Daily use, Trucks percent use, Age, Design and Material are all vital variables in order to gauge a clear relationship.  

The coefficient of determination, R2, is used to analyse how the differences in a target variable can be explained by a difference in the predictor variables.

More specifically, R-squared gives the percentage variation in y explained by x-variables. The range is 0 to 1.

From the below calculation is can be found that R-squared gives the degree of variability in the target variable that is explained by the model or the independent variables. With an R2 value of 0.451, then it means that the predictor variables explain 45.1% of the variation in the target variable.

The Y-intercept represents the value of y when x = 0. On an x, y graph, a Y-intercept is the part of the curve that crosses the y-axis which gives a value of 24.

In this simple linear regression, the size of the coefficient for each independent predictor variable gives the size of the effect that a variable is having on your dependent variable, and the sign on the coefficient whether it be positive or negative gives the direction of the effect. In this regression, the coefficients for **Age**, **Mat Other**, **Mat Steel**, **Des Beam**, **Des Other**, **Des Slab**, **Des Truss** tells us that the dependent variable is expected to decrease by 0.049, 2.78, 1.35, 1.51, 1.53, 1.597, 0.826 respectively. However, **AverageDaily** and **Truck_percent** When the independent variable increases by one AverageDaily has no change while Truck percent increase by 0.005. As multiple independent variables are being used, the coefficient tells you how much the dependent variable is expected to increase/decrease when that independent variable increases by one, holding all the other independent variables constant. 


The Root Mean Squared Error which indicates that the observed data are close to each other shows a better accuracy. Thus a lower RMSE show a better model performance. As the RMSE is 1.45 making it a bad fit.



In [ ]:
y = bridges_five_pred1.Current_Condition # this is the target variable; we assue just one though more are possible
X = np.column_stack((bridges_five_pred1.Age, bridges_five_pred1.AverageDaily, bridges_five_pred1.Trucks_percent, Mat_D.Other, Mat_D.Steel, Des_D.Beam, Des_D.Other, Des_D.Slab, Des_D.Truss))
reg = LinearRegression().fit(X, y)

In [ ]:
reg = LinearRegression().fit(X, y)
print('The R2 coefficient of determination is %4.3f' % reg.score(X, y))
print('The intercept is %4.1f' % reg.intercept_)
print("The regression coefficients are: " + str(reg.coef_.round(4)))


In [ ]:
data = [["Age", -0.0486], ["AverageDaily", -0], ["Trucks_percent", 0.005], ["Mat Other", -2.7801],["Mat Steel", -1.3512], ["Des Beam", -1.5157], ["Des Other", -1.5313], ["Des Slab", -1.5966], ["Des Truss", -0.8264]]
df = pd.DataFrame(data, columns = ['Variables', 'Regression Coefficient'])
df

The larger the coefficient the greater the effect it has on the target variable

In [ ]:
y_hat = reg.predict(X)

fig, a1 = plt.subplots(1, 1)
residuals = y_hat - y
a1.hist(residuals, bins=10, density=True)
_ = a1.set_xlabel('Error in prediction: predicted - actual current condition')

In [ ]:
print('RMSE: %.2f' % mean_squared_error (y, y_hat, squared=False))

Below is a scatter plot for the  actual value against the predicted values. If the regression fitted better, the points would be nearer to the blue line. Although there is some evidence of a positive correlation but very minor.

In [ ]:
fig, a = plt.subplots(1,1,figsize=(15,10))
a.scatter(y_hat, y,  color='black')
a.plot(y_hat, y_hat, color='blue', linewidth=3)

a.set_xlabel('Predicted Value')
a.set_ylabel('Actual Value')

Conclusion - best to include more variables to create a more detailed are more accurate set of results